# Extract Logic

##### this file is demo file for prototyping python functions

In [ ]:
import json
import requests
import random
api_key = '3a3bd8764d180fafd01e23bafc751a09'
base_url= 'https://v3.football.api-sports.io/'



In [ ]:
def fetch_fixtures():


    endpoint = "/fixtures"
    url =f"{base_url+endpoint}"
    querystring = {"date":"2025-06-08"}

    headers = {'x-rapidapi-host': 'v3.football.api-sports.io',
                 "x-rapidapi-key":api_key
               }

    response = requests.get(url, headers=headers, params=querystring)
    print(response.status_code)
    print(json.dumps(response.json(), ensure_ascii=False, indent=2))
    
    return   response.json()["response"]


In [ ]:
fixtures=fetch_fixtures()

In [ ]:
limited_fixtures = random.sample(fixtures, 100)  # pick 100 unique random fixtures
#len(fixtures)

In [ ]:
def  extract_teams_id(fixtures):
    teams_ids=set()
    metadata=[]
    
    for fixture in fixtures:
        home_team = fixture["teams"]["home"]["id"]
        away_team = fixture["teams"]["away"]["id"]
        league_id = fixture["league"]["id"]
        season = fixture["league"]["season"]

        teams_ids.update([home_team, away_team])

        metadata.append({
            "home_team": home_team,
            "away_team": away_team,
            "league_id": league_id,
            "season": season
        })
        # print(list(teams_ids))

    return list(teams_ids), metadata 
    



In [ ]:
teams_IDs,metadata=extract_teams_id(limited_fixtures)

In [ ]:
print(len(teams_IDs))
print(len(limited_fixtures))
print(len(metadata))

In [ ]:
print(teams_IDs[:10])

#### **build team requests**

In [ ]:
def build_team_requests(teams, metadata):
    team_requests = set()

    for match in metadata:
        for team_id in [match["home_team"], match["away_team"]]:
            if team_id in teams:
                team_requests.add((
                    team_id,
                    match["league_id"],
                    match["season"]
                ))

    return list(team_requests)

In [ ]:
team_requests=build_team_requests(teams_IDs,metadata)

In [ ]:
print(len(team_requests))

In [ ]:
def fetch_team_info(team_id):
    endpoint = "/teams"
    url =f"{base_url+endpoint}"
    headers = {'x-rapidapi-host': 'v3.football.api-sports.io',
                 "x-rapidapi-key":api_key
               }

    querystring = {"id":team_id}

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed for team {team_id}")
        return None

In [ ]:
import time

def extract_team_info_from_fixtures(team_requests):
    all_team_stats = []

    for team_id in team_requests:
        info = fetch_team_info(team_id)
        if info:
            all_team_stats.append(info)

        time.sleep(6)  # ⏱️ Wait 6 seconds to respect 10 reqs/minute

    return all_team_stats


In [ ]:
team_info = extract_team_info_from_fixtures(teams_IDs)

In [ ]:
print(len(team_info))
team_info[:10]

---

# Transfer Logic

### Matches Transfom

In [ ]:
import pandas as pd

In [ ]:

match_data = []
for f in fixtures:
    match_data.append({
        'match_id': f['fixture']['id'],
        'date': f['fixture']['date'],
        'match_status':f['fixture']['status']['short'],
        'home_team_id': f['teams']['home']['id'],
        'away_team_id': f['teams']['away']['id'],
        'home_team_score':f['score']['fulltime']['home'],
        'away_team_score':f['score']['fulltime']['away'],
        'venue': f['fixture']['venue']['name'],
        'league_id': f['league']['id'],
        'league':f['league']['name'],
        'country':f['league']['country'],
        'season': f['league']['season']
    })
match_df = pd.DataFrame(match_data)
#df['date'] = pd.to_datetime(df['date'])


In [ ]:
# Data Cleansing 
# 
#  Null values handling:
match_df = match_df.dropna(subset=['match_id'])
match_df = match_df.dropna(subset=['home_team_id'])
match_df = match_df.dropna(subset=['away_team_id'])
match_df.drop(match_df[match_df['match_status']=='CANC'].index,inplace=True)
match_df = match_df.dropna(subset=['date'])
match_df = match_df.dropna(subset=['season'])
match_df = match_df.dropna(subset=['league_id'])

#changing the time format
match_df['date'] =pd.to_datetime(match_df['date'])


match_df['country']=match_df['country'].fillna('Unknown')
match_df['match_status']=match_df['match_status'].fillna('Unknown')
match_df['home_team_score']=match_df['home_team_score'].fillna(0)
match_df['away_team_score']=match_df['away_team_score'].fillna(0)
match_df['venue']=match_df['venue'].fillna('Unknown')


#-------------------------------------------------------
match_df['venue']=match_df['venue'].astype('string')
match_df['country']=match_df['country'].astype('string')
match_df['league']=match_df['league'].astype('string')
match_df['match_status']=match_df['match_status'].astype('string')
match_df['home_team_score']=match_df['home_team_score'].astype('int')
match_df['away_team_score']=match_df['away_team_score'].astype('int')
match_df['away_team_score']=match_df['away_team_score'].astype('int')

In [ ]:
match_df

In [ ]:
match_df.dtypes

In [ ]:
""" 
checking null values 
"""
match_df.isnull().sum()


### Teams Transfrom

In [208]:

# Your JSON string (as shown earlier)
json_string = """
{
  "get": "teams",
  "parameters": {
    "id": "33"
  },
  "errors": [],
  "results": 1,
  "paging": {
    "current": 1,
    "total": 1
  },
  "response": [
    {
      "team": {
        "id": 33,
        "name": "Manchester United",
        "code": "MUN",
        "country": "England",
        "founded": 1878,
        "national": false,
        "logo": "https://media.api-sports.io/football/teams/33.png"
      },
      "venue": {
        "id": 556,
        "name": "Old Trafford",
        "address": "Sir Matt Busby Way",
        "city": "Manchester",
        "capacity": 76212,
        "surface": "grass",
        "image": "https://media.api-sports.io/football/venues/556.png"
      }
    }
  ]
}
"""
json_response=json.loads(json_string)

In [217]:
team_datas=[{
    'team_id':t['team']['id'],
    'team_name':t['team']['name'],
    'team_code':t['team']['code'],
    'country':t['team']['country'],
    'city':t['venue']['city'],
    'founded':t['team']['founded'],
    'venue_id':t['venue']['id'],
    'venue_name':t['venue']['name'],

    }for t in json_response['response']]

In [219]:
team_datas
df=pd.DataFrame(team_datas)

In [220]:
df

,team_id,team_name,team_code,country,city,founded,venue_id,venue_name
0,33,Manchester United,MUN,England,Manchester,1878,556,Old Trafford


In [222]:
df.dropna(subset=['team_id','team_name','venue_id'],inplace=True)
df.fillna({'team_code':'UNK','country':'Unknown','city':'Uknown','founded':0,'venue_name':'Unkown'},inplace=True)

In [223]:
df.dtypes

team_id        int64
team_name     object
team_code     object
country       object
city          object
founded        int64
venue_id       int64
venue_name    object
dtype: object

In [ ]:
json_string=
"""
{
  "get": "players/squads",
  "parameters": {
    "team": "33"
  },
  "errors": [],
  "results": 1,
  "paging": {
    "current": 1,
    "total": 1
  },
  "response": [
    {
      "team": {
        "id": 33,
        "name": "Manchester United",
        "logo": "https://media.api-sports.io/football/teams/33.png"
      },
      "players": [
        {
          "id": 882,
          "name": "David de Gea",
          "age": 31,
          "number": 1,
          "position": "Goalkeeper",
          "photo": "https://media.api-sports.io/football/players/882.png"
        },
        {
          "id": 883,
          "name": "L. Grant",
          "age": 38,
          "number": 13,
          "position": "Goalkeeper",
          "photo": "https://media.api-sports.io/football/players/883.png"
        },
        {
          "id": 2931,
          "name": "T. Heaton",
          "age": 35,
          "number": null,
          "position": "Goalkeeper",
          "photo": "https://media.api-sports.io/football/players/2931.png"
        },
        {
          "id": 19088,
          "name": "D. Henderson",
          "age": 24,
          "number": 26,
          "position": "Goalkeeper",
          "photo": "https://media.api-sports.io/football/players/19088.png"
        },
        {
          "id": 885,
          "name": "E. Bailly",
          "age": 27,
          "number": 3,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/885.png"
        },
        {
          "id": 886,
          "name": "Diogo Dalot",
          "age": 22,
          "number": 20,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/886.png"
        },
        {
          "id": 153434,
          "name": "W. Fish",
          "age": 18,
          "number": 48,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/153434.png"
        },
        {
          "id": 888,
          "name": "P. Jones",
          "age": 29,
          "number": 4,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/888.png"
        },
        {
          "id": 138775,
          "name": "E. Laird",
          "age": 20,
          "number": null,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/138775.png"
        },
        {
          "id": 2935,
          "name": "H. Maguire",
          "age": 28,
          "number": 5,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/2935.png"
        },
        {
          "id": 889,
          "name": "V. Lindelöf",
          "age": 27,
          "number": 2,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/889.png"
        },
        {
          "id": 891,
          "name": "L. Shaw",
          "age": 26,
          "number": 23,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/891.png"
        },
        {
          "id": 378,
          "name": "Alex Telles",
          "age": 29,
          "number": 27,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/378.png"
        },
        {
          "id": 19182,
          "name": "A. Tuanzebe",
          "age": 24,
          "number": 38,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/19182.png"
        },
        {
          "id": 18846,
          "name": "A. Wan-Bissaka",
          "age": 24,
          "number": 29,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/18846.png"
        },
        {
          "id": 138806,
          "name": "B. Williams",
          "age": 21,
          "number": 33,
          "position": "Defender",
          "photo": "https://media.api-sports.io/football/players/138806.png"
        },
        {
          "id": 1485,
          "name": "Bruno Fernandes",
          "age": 27,
          "number": 18,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/1485.png"
        },
        {
          "id": 895,
          "name": "J. Garner",
          "age": 20,
          "number": null,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/895.png"
        },
        {
          "id": 899,
          "name": "Andreas Pereira",
          "age": 25,
          "number": 15,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/899.png"
        },
        {
          "id": 900,
          "name": "J. Lingard",
          "age": 29,
          "number": 14,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/900.png"
        },
        {
          "id": 901,
          "name": "Mata",
          "age": 33,
          "number": 8,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/901.png"
        },
        {
          "id": 902,
          "name": "N. Matić",
          "age": 33,
          "number": 31,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/902.png"
        },
        {
          "id": 903,
          "name": "S. McTominay",
          "age": 25,
          "number": 39,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/903.png"
        },
        {
          "id": 180560,
          "name": "H. Mejbri",
          "age": 18,
          "number": 46,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/180560.png"
        },
        {
          "id": 904,
          "name": "P. Pogba",
          "age": 28,
          "number": 6,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/904.png"
        },
        {
          "id": 905,
          "name": "Fred",
          "age": 28,
          "number": 17,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/905.png"
        },
        {
          "id": 163054,
          "name": "S. Shoretire",
          "age": 17,
          "number": 74,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/163054.png"
        },
        {
          "id": 547,
          "name": "D. van de Beek",
          "age": 24,
          "number": 34,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/547.png"
        },
        {
          "id": 138814,
          "name": "E. Galbraith",
          "age": 20,
          "number": null,
          "position": "Midfielder",
          "photo": "https://media.api-sports.io/football/players/138814.png"
        },
        {
          "id": 274,
          "name": "E. Cavani",
          "age": 34,
          "number": 7,
          "position": "Attacker",
          "photo": "https://media.api-sports.io/football/players/274.png"
        },
        {
          "id": 153430,
          "name": "A. Elanga",
          "age": 19,
          "number": 56,
          "position": "Attacker",
          "photo": "https://media.api-sports.io/football/players/153430.png"
        },
        {
          "id": 897,
          "name": "M. Greenwood",
          "age": 20,
          "number": 11,
          "position": "Attacker",
          "photo": "https://media.api-sports.io/football/players/897.png"
        },
        {
          "id": 19329,
          "name": "D. James",
          "age": 24,
          "number": 21,
          "position": "Attacker",
          "photo": "https://media.api-sports.io/football/players/19329.png"
        },
        {
          "id": 908,
          "name": "A. Martial",
          "age": 26,
          "number": 9,
          "position": "Attacker",
          "photo": "https://media.api-sports.io/football/players/908.png"
        },
        {
          "id": 909,
          "name": "M. Rashford",
          "age": 24,
          "number": 10,
          "position": "Attacker",
          "photo": "https://media.api-sports.io/football/players/909.png"
        },
        {
          "id": 157997,
          "name": "A. Diallo",
          "age": 19,
          "number": 19,
          "position": "Attacker",
          "photo": "https://media.api-sports.io/football/players/157997.png"
        }
      ]
    }
  ]
}"""

### Transform Squads

In [235]:
json_response=json.loads(json_string)
#json_response=json_response['response']

In [246]:
print(json_response.get('response'))


[{'team': {'id': 33, 'name': 'Manchester United', 'code': 'MUN', 'country': 'England', 'founded': 1878, 'national': False, 'logo': 'https://media.api-sports.io/football/teams/33.png'}, 'venue': {'id': 556, 'name': 'Old Trafford', 'address': 'Sir Matt Busby Way', 'city': 'Manchester', 'capacity': 76212, 'surface': 'grass', 'image': 'https://media.api-sports.io/football/venues/556.png'}}]


In [ ]:
squad = []

for s in json_response.get('response', []):
    team = s.get('team')
    players = s.get('players')

    if team and players:
        team_id = team.get('id')
        for player in players:
            squad.append({
                'team_id': team_id,
                'player_id': player.get('id'),
                'name': player.get('name'),
                'age': player.get('age'),
                'number': player.get('number'),
                'photo': player.get('photo'),
                'position': player.get('position')
            })

print(squad)

[]


In [251]:
squad

[]